In [2]:
from Pianki.Analiza_pianek import saldo, naliczone, wstrzymane
# from Pianki.Analiza_pianek.przygotowanie_danych import aktualizuj_saldo

# aktualizuj_saldo()
# from Modele_db import engine

# with engine.begin() as conn:
#     saldo.rename(columns={"KOD": "kod", "SALDO": "stan"}).to_sql("SALDO", conn, if_exists="replace")
#     naliczone.rename(columns={'LIMIT_NAZWA':"limit_nazwa", 'KOD':"kod", 'ZAPOT_ZLEC':"zapot_zlec", 'LIMIT_DATA_PROD':"limit_data_prod"}).to_sql("NALICZONE", conn, if_exists="replace")
#     wstrzymane.rename(columns={"KOD":"kod", 'ILOSC':"ilosc"}).to_sql("WSTRZYMANE", conn, if_exists="replace")

In [1]:
from Modele_db.modele_db import *
from Pianki.Analiza_pianek.funkcje_analizy_pianek import *

# pd.set_option()

In [ ]:
Ogolna_analiza_objetosci()
print("---\nZAPEŁNIENIE MAGAZYNU")
Ogolna_analiza_objetosci("podsum_prc")

In [ ]:
from Modele_db.modele_db import session, ZAM_PIANKI, KOMPLETY_PIANEK
from sqlalchemy import func


session.query(func.sum(ZAM_PIANKI.ile_zam * KOMPLETY_PIANEK.obj)).join(KOMPLETY_PIANEK, KOMPLETY_PIANEK.kod == ZAM_PIANKI.kod).filter(ZAM_PIANKI.data_zakonczenia.between(dt(2024,9,29), dt(2024,10,6))).all()


In [7]:
br = Braki(WOLNE="SALDO")[0] #USUNAC PROBLEM Z DUBLUJACYMI SIE ZAMOWIANIAMI(JEDNO DOSTARCZONE DRUGIE W DRODZE TWORZĄ SIĘ DWIE POZYCJE)
br.sort_values(by=["GRUPA", "OPIS"]).drop_duplicates("OPIS")
# br.to_excel("BRAKI.xlsx")

In [ ]:
Zagrozone(WOLNE="SALDO")
zgr = Zagrozone(False,WOLNE="SALDO")
# zgr
# zgr[((zgr.ZAMOWIONE + zgr.CZEKA_NA_SPAKOWANIE) == 0)]
# zgr[(zgr.SALDO < zgr.MIN)]
zgr[((zgr.ZAMOWIONE + zgr.CZEKA_NA_SPAKOWANIE) == 0)&(zgr.SALDO < zgr.MIN)]

In [1]:
#IMPORT ANAMLIZY PIANEK
from Pianki.Analiza_pianek.instrukcje_zamawiana import instrukcja_zamawiania_pianpol as izp, instrukcja_zamawiania_ciech as izc
from Pianki.Analiza_pianek.Podsumowanie_analizy_pianek import Podsumowanie_analizy_pianek

arp = Podsumowanie_analizy_pianek(izp)
# arc = Podsumowanie_analizy_pianek(izc)

VOL 0!! ELI_MIS


In [2]:
arp._Tabela_podsumowania_analizy

,GRUPA,MODEL,POZ_ZAGR,POZ_ZAGR_NIE_ZAM,BRAKI,ILOSC_BRAKOW,BRYL_DO_ZAMOWIENIA,WSPL_DO_ZAM,OBJ_CIECH,OBJ_VITA,OBJ_PIANPOL,*WSPL_ZAP_WOLNE,*WSPL_ZAP_ZAM
0,1,AVANT,9,1,5,110.0,88.0,10.0,0,0.00,22.01,16.0,96.0
2,1,STONE,10,0,0,0.0,75.0,10.0,0,20.11,4.18,39.0,94.0
3,1,HORIZON,5,0,1,1.0,84.0,8.0,0,21.87,4.69,69.0,95.0
1,1,ELIXIR,7,0,0,0.0,17.0,4.0,0,0.00,6.63,29.0,98.0
7,2,OXYGEN,3,3,0,0.0,66.0,33.0,0,0.00,21.76,71.0,71.0
5,2,DIVA,5,5,1,1.0,72.0,30.0,0,0.00,24.48,70.0,71.0
9,2,REVERSO,1,1,0,0.0,185.0,23.0,0,33.79,6.63,78.0,78.0
6,2,MAXWELL,3,1,0,0.0,50.0,20.0,0,0.00,13.99,71.0,83.0
4,2,AMALFI,0,0,0,0.0,73.0,16.0,0,0.00,31.44,84.0,84.0
8,2,WILLOW,3,0,0,0.0,15.0,4.0,0,0.00,7.11,83.0,109.0


In [ ]:
ava = arp["AVANT"].ar
ava.DO_ZAM_SZT.mean()

In [ ]:
analiza[analiza.OPIS.str.contains("WIL")][['index', 'KOD', 'OPIS', 'MAX','SALDO',
       'ZLECENIA', 'WST', 'CZEKA_NA_SPAKOWANIE', 'ZAMOWIONE',
       'CZESIOWO_DOSTARCZONE', '29/01', '30/01', 'MIN', 'SUMA_ZLEC',
       'SALDO_Z_NIE_SPAK', 'WOLNE_SALDO', 'WOLNE_NIE_SPAK', 'DO_ZAM_SZT',
       ]]

In [ ]:
ava[ava.DO_ZAM_SZT > ava.MIN]

In [ ]:
(arp > 1).query("GRUPA < 4")

In [ ]:
arp.Tabela_podsumowania_analizy#.query("GRUPA == 4")

In [ ]:
arp["STONE"].ar

In [ ]:
"sqlite:///Z:/450. PLANISTA - ZAOPATRZENIE/ARCHIWUM_ANALIZ.db"



In [ ]:
arp._Tabela_podsumowania_analizy.columns

In [4]:
Zapis_danych_do_Archiwum(243m, analiza, arp._Tabela_podsumowania_analizy, "sqlite:///Z:/450. PLANISTA - ZAOPATRZENIE/ARCHIWUM_ANALIZ.db")